In [19]:
import pandas as pd
import scipy as ss
import seaborn as sns
import numpy as np

Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

- В выгрузке только уникальные действия пользователей за каждый день 

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

In [20]:
#Приведем колонку дата к временному типу
df = pd.read_csv('KC_case_data .csv', parse_dates=['date'])

Посмотрим на данные, проверим на наличие пропусков

In [21]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [22]:
df.isna().sum()

date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64

In [23]:
df.dtypes

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object

In [24]:
df.shape

(2747968, 8)

MAU в феврале

In [25]:
#Извлекаем из даты месяц, и создаем новую колонку в дф month
df['month'] = df['date'].dt.month

In [26]:
february = df.query('month == 2')

In [27]:
february.query('event == "app_start"')\
    .agg({'device_id' : 'unique'})

,device_id
0,27421
1,52191
2,57689
3,61236
4,118707
...,...
75027,7429766
75028,17946030
75029,918311
75030,1231749


За февраль 75032 уникальных пользователя, кто хотя бы открывал приложение за месяц

Количество установок в январе:

In [28]:
january = df.query('month == 1')

In [29]:
january.query('event == "app_install"')\
    .agg({'device_id' : 'unique'})

,device_id
0,4921563
1,1311583
2,23006325
3,5645652
4,8798567
...,...
80292,3606659
80293,11501360
80294,10118705
80295,19855521


Присвоим пользователям когорты по дню установки приложения и посчитаем для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей?
Считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней

In [33]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,1
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,1
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,1
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,1
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,1


In [34]:
#Отбираем пользователей, которые установили приложение, и отбираем колонку дата и девайс айди
installs = df[df['event'] == "app_install"][['date', 'device_id']]

In [36]:
#убираем дубликаты device id, чтобы остались только уникальные пользователи
installs = installs.drop_duplicates('device_id')

In [38]:
#переименовываем колонку дата в дата установки
installs = installs.rename(columns={'date': 'install_date'})

In [40]:
#Отбираем пользователей по покупке
purchase = df[df['event'] == 'purchase'][['date', 'device_id']]

In [42]:
#убираем дубликаты device id, чтобы остались только уникальные пользователи
purchase = purchase.drop_duplicates('device_id')

In [44]:
#Переименовываем колонку дата в дата первой покупки
purchase = purchase.rename(columns={'date': 'first_buy_date'})

In [46]:
#Джойним датафрейм по установкам с дф с покупками
data = installs.merge(purchase, how='left', on='device_id')

In [48]:
#Вычитаем из колонки первая покупка колонку дата установки
#Тем самым видем сколько дней прошло между покупкой и установкой
data['date_diff'] = data['first_buy_date'] - data['install_date']

In [50]:
#По нашему условию, мы считаем сконвертировавшегося пользователя, 
#Если между устаовкой и первой покупкой прошло не более 7 дней
data_7 = data.query('date_diff <= "7 days"')

In [62]:
#Группируемся по дате и считаем кол-во уникаьных пользователей
#Кто по условию <= 7 дней
payers = data_7.groupby('install_date', as_index=False)\
    .agg({'device_id': 'nunique'})\
    .rename(columns={'install_date' : 'date'})\
    .rename(columns={'device_id' : 'payers'})

In [66]:
#Считаем уникальных пользователей кто установил приложение
users = data.groupby('install_date', as_index=False)\
    .agg({'device_id': 'nunique'})\
    .rename(columns={'install_date' : 'date'})\
    .rename(columns={'device_id' : 'users'})

In [68]:
# Обьединяем уникальных пользователей, кто установил приложение
# с покупателями(у кого между установкой и первой покупкой прошло <= 7 дней)

dff = users.merge(payers, how='left', on='date')

In [72]:
#считаем конверсию из установивших в совершивших покупку
dff['CR'] = dff.payers / dff.users

In [73]:
dff

,date,users,payers,CR
0,2020-01-01,3579,1408,0.393406
1,2020-01-02,3144,1186,0.377226
2,2020-01-03,2402,834,0.347211
3,2020-01-04,1831,639,0.348990
4,2020-01-05,1671,587,0.351287
...,...,...,...,...
86,2020-03-27,1199,277,0.231026
87,2020-03-28,1091,209,0.191567
88,2020-03-29,1117,223,0.199642
89,2020-03-30,994,204,0.205231


С какого платного маркетингового канала пришло больше всего новых пользователей? 

In [82]:
df.query('event == "app_install"')\
    .groupby('utm_source')\
    .agg({'device_id' : 'nunique'})\
    .sort_values('device_id', ascending=False)

,device_id
utm_source,
-,32460
yandex-direct,29368
google_ads,26286
vk_ads,23189
instagram_ads,20096
facebook_ads,13916
referal,9282


На каком этапе воронки отваливается бОльшая часть клиентов. Посмотрим отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

In [83]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,1
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,1
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,1
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,1
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,1


In [84]:
#Отбираем пользователей, кто зарегался и оставляем от основного дф
#колонку дата и пользователей
register = df[df['event'] =="register"][['date', 'device_id']]

In [85]:
#переименовываем колонку дата в дата регистрации
register = register.rename(columns={'date' : 'reg_date'})

In [86]:
#Получили дф с уникальными пользователями и датой их регистрации

register

,reg_date,device_id
2669658,2020-01-01,294193
2669659,2020-01-01,22917617
2669660,2020-01-01,15248490
2669661,2020-01-01,252062
2669662,2020-01-01,2251583
...,...,...
2747963,2020-03-31,2984778
2747964,2020-03-31,27301864
2747965,2020-03-31,1294285
2747966,2020-03-31,3010574


In [87]:
#Делаем новый дф для работы 
#Джойним основной дф с дф пользователей и датой их регистрации
df_9 = df.merge(register, how='left', on='device_id')

In [89]:
#Отбираем пользователей у которых дата реги меньше какого либо действия
#Тем самым понимаем, что они зарегистриванные пользователи
df_reg = df_9[df_9['reg_date'] < df_9['date']]

In [92]:
#Группируемся по действию и выводим кол-во уникальных пользователей, которые совершили какое либо действие
df_reg.groupby('event', as_index=False)\
    .agg({'device_id' : 'nunique'})

,event,device_id
0,app_start,40991
1,choose_item,37926
2,purchase,24880
3,search,40482
4,tap_basket,34517


Считаем конверсию каждого этапа воронки

In [93]:
print("Конверсия в поиск из открытия", np.round((40482 / 40991)*100, 2))
print("Конверсия из добавления товара из поиска", np.round((37926 / 40482)*100, 2))
print("Конверсия перехода в корзину из добавления товара", np.round((34517 / 37926)*100, 2))
print("Конверсия в покупку из корзины", np.round((24880 / 34517)*100, 2))

Конверсия в поиск из открытия 98.76
Конверсия из добавления товара из поиска 93.69
Конверсия перехода в корзину из добавления товара 91.01
Конверсия в покупку из корзины 72.08


Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?


In [107]:
#Отбираем пользователей по покупке
first_buy = df.query('event == "purchase"')\
    .groupby('utm_source')\
    .agg({'device_id': 'nunique'})\
    .sort_values('device_id', ascending=False)

In [108]:
first_buy

,device_id
utm_source,
-,16598
vk_ads,12364
yandex-direct,12028
google_ads,11339
instagram_ads,10762
facebook_ads,9017
referal,6362


In [109]:
first_start = df.query('event == "app_start"')\
    .groupby('utm_source')\
    .agg({'device_id' : 'nunique'})\
    .sort_values('device_id', ascending=False)

In [110]:
first_start

,device_id
utm_source,
-,52268
yandex-direct,40707
google_ads,38092
vk_ads,34079
instagram_ads,31045
facebook_ads,25957
referal,15924


In [112]:
CR = first_buy.device_id / first_start.device_id
CR.sort_values(ascending=False)

utm_source
referal          0.399523
vk_ads           0.362804
facebook_ads     0.347382
instagram_ads    0.346658
-                0.317556
google_ads       0.297674
yandex-direct    0.295477
Name: device_id, dtype: float64

Пользователи, пришедшие с какого канала, имеют медианный первый чек выше?

In [128]:
#Отбираем пользователей по покупке
purchases = df[df['event'] == "purchase"][['date', 'device_id', 'purchase_sum', 'utm_source']]
purchases = purchases.sort_values('date').drop_duplicates('device_id')

In [130]:
purchases.groupby('utm_source').median()

,device_id,purchase_sum
utm_source,,
-,10972119.0,398.5
facebook_ads,10844491.0,389.0
google_ads,11089575.0,390.5
instagram_ads,11096726.0,393.5
referal,10491527.0,395.5
vk_ads,11334978.0,393.0
yandex-direct,10969061.5,392.5


Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI? 

In [131]:
revenue = df.groupby('utm_source')\
    .agg({'purchase_sum' : 'sum'})

In [132]:
revenue

,purchase_sum
utm_source,
-,21449749.5
facebook_ads,12249901.0
google_ads,12868276.0
instagram_ads,14546969.0
referal,8837044.5
vk_ads,16389652.5
yandex-direct,13915368.0


In [133]:
#Добавляем колонку с затратами на рекламу
revenue['marketing_cost'] = [0, 8590498, 10534878, 8561626, 0, 9553531, 10491707]

In [134]:
revenue

,purchase_sum,marketing_cost
utm_source,,
-,21449749.5,0
facebook_ads,12249901.0,8590498
google_ads,12868276.0,10534878
instagram_ads,14546969.0,8561626
referal,8837044.5,0
vk_ads,16389652.5,9553531
yandex-direct,13915368.0,10491707


In [135]:
revenue['ROMI'] = (revenue['purchase_sum'] - revenue['marketing_cost']) / revenue['marketing_cost']

In [137]:
revenue.sort_values('ROMI', ascending=False)

,purchase_sum,marketing_cost,ROMI
utm_source,,,
-,21449749.5,0,inf
referal,8837044.5,0,inf
vk_ads,16389652.5,9553531,0.715560
instagram_ads,14546969.0,8561626,0.699090
facebook_ads,12249901.0,8590498,0.425983
yandex-direct,13915368.0,10491707,0.326321
google_ads,12868276.0,10534878,0.221493
